In [ ]:
# Cell 1: Import Libraries

import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
import random
import math
import re
import time
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.visualize import display_instances
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib, utils
import pandas as pd
from pathlib import Path
import shutil
import yaml

In [ ]:
# Cell 2: Load Data

# Load Train and Test Data

#note change it to ur path in ur laptop 
train = pd.read_csv('zindi ml/Train.csv') 
test = pd.read_csv('zindi ml/Test.csv')

# Map str classes to ints (label encoding targets)
class_mapper = {x:y for x,y in zip(sorted(train['class'].unique().tolist()), range(train['class'].nunique()))}
train['class_id'] = train['class'].map(class_mapper)

# Split data into training and validation
from sklearn.model_selection import train_test_split
train_unique_imgs_df = train.drop_duplicates(subset = ['Image_ID'], ignore_index = True)
X_train, X_val = train_test_split(train_unique_imgs_df, test_size = 0.25, stratify=train_unique_imgs_df['class'], random_state=42)

X_train = train[train.Image_ID.isin(X_train.Image_ID)]
X_val = train[train.Image_ID.isin(X_val.Image_ID)]

In [ ]:
# Cell 3: Prepare Image Directories

# Create Directories for Images
DATA_DIR = Path('zindi ml/')  # this is like ur main directory 
DATASET_DIR = Path('zindi ml/datasets')
IMAGES_DIR = DATASET_DIR / 'images'
TRAIN_IMAGES_DIR = IMAGES_DIR / 'train'
VAL_IMAGES_DIR = IMAGES_DIR / 'val'
TEST_IMAGES_DIR = IMAGES_DIR / 'test'

# Copy Images to Respective Directories
for img in X_train.image_path.unique():
    shutil.copy(img, TRAIN_IMAGES_DIR / img.parts[-1])

for img in X_val.image_path.unique():
    shutil.copy(img, VAL_IMAGES_DIR / img.parts[-1])

for img in test.image_path.unique():
    shutil.copy(img, TEST_IMAGES_DIR / img.parts[-1])

In [ ]:
# Cell 4: Create Data.yaml (Not Required for Mask R-CNN, but for Consistency)

class_names = sorted(train['class'].unique().tolist())
num_classes = len(class_names)
data_yaml = {
    'train': '/content/' + str(TRAIN_IMAGES_DIR),
    'val': '/content/' + str(VAL_IMAGES_DIR),
    'test': '/content/' + str(TEST_IMAGES_DIR),
    'nc': num_classes,
    'names': class_names
}
yaml_path = 'data.yaml'
with open(yaml_path, 'w') as file:
    yaml.dump(data_yaml, file, default_flow_style=False)

In [ ]:
# Cell 5: Note - Train Model using custom.py  
# open terminal in directory where custom.py is in run 
# python custom.py ......before u must have requirements so pip install -r requirements.txt first

In [ ]:
# Cell 6: Load Trained Model and Make Predictions

inference_config = Config()
inference_config.set_class_names(class_names)

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir='zindi ml/logs')



In [ ]:
# Load trained weights 
model_path = os.path.join('zindi ml/logs', "mask_rcnn_object_0100.h5")
assert model_path.exists(), "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)



In [ ]:
# Initialize Variables for Prediction Output
all_data = []



In [ ]:
# Iterate Through Each Image in Test Directory
test_dir_path = 'zindi ml/datasets/images/test'
image_paths = []
for filename in os.listdir(test_dir_path):
    if os.path.splitext(filename)[1].lower() in ['.png', '.jpg', '.jpeg']:
        image_paths.append(os.path.join(test_dir_path, filename))

for image_path in image_paths:
    img = skimage.io.imread(image_path)
    img_arr = np.array(img)
    results = model.detect([img_arr], verbose=1)
    r = results[0]
    for box, cls, conf, mask in zip(r['rois'], r['class_ids'], r['scores'], r['masks']):
        x1, y1, x2, y2 = box
        detected_class = class_names[int(cls) - 1]  # Adjusted Index
        all_data.append({
            'Image_ID': os.path.basename(image_path),
            'class': detected_class,
            'confidence': conf,
            'ymin': y1,
            'xmin': x1,
            'ymax': y2,
            'xmax': x2
        })



In [ ]:
# Convert List to DataFrame
sub = pd.DataFrame(all_data)  



In [ ]:
# Save to Submission CSV
sub.to_csv('submission.csv', index = False)